# Map the competitive routes

10 most promising corridors to support (overall across operators), leave it in a separate notebook. in urban districts, strong pop signal where it could be served, which are the 10 most promising corridors.

In [1]:
import branca
import geopandas as gpd
import intake
import os
import numpy as np
import pandas as pd

import utils
from shared_utils import map_utils, geography_utils
from shared_utils import calitp_color_palette as cp

catalog = intake.open_catalog("./*.yml")

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0427 16:58:44.522552070    1231 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
df = pd.read_parquet("./data/stripplot_trips.parquet")

In [21]:
keep_cols = [
    "calitp_itp_id", "route_id", "p25", "p50", "p75",
    "car_duration_hours",
    "pct_trips_competitive", "num_trips", "num_competitive",
    "bus_multiplier_spread", "bus_difference_spread",
    "caltrans_district", 
]

df2 = (df[(df.service_hours >= df.p25) & 
          (df.service_hours <= df.p75) &
          (df.service_hours >= 0.75)]
       [keep_cols].drop_duplicates()
       .sort_values(["pct_trips_competitive", "bus_multiplier_spread", 
                     "num_competitive",
                     "p50"],
                    ascending=[False, True, False, True]
                   )
       .reset_index(drop=True)
      )

In [22]:
gdf = catalog.gmaps_results.read()

gdf = pd.merge(
    gdf[["calitp_itp_id", "shape_id", "route_id", "geometry"]],
    df2,
    on = ["calitp_itp_id", "route_id"],
    how = "inner",
    validate = "1:1",
)


In [23]:
# Need a numeric identifier, or branca.colormap will error
gdf = gdf.assign(
    # Try to sort/group slightly differently or else colors for same operator appear too similar
    identifier2 = gdf.apply(lambda x: np.random.randint(0, 1619), axis=1),
)


gdf = (gdf.sort_values(
    ["pct_trips_competitive", "bus_multiplier_spread", 
     "num_competitive", "p50"],
    ascending=[False, True, False, True])
       .reset_index(drop=True)
       .assign(order = gdf.index)
)

In [24]:
gdf.head(10)

,calitp_itp_id,shape_id,route_id,geometry,p25,p50,p75,car_duration_hours,pct_trips_competitive,num_trips,num_competitive,bus_multiplier_spread,bus_difference_spread,caltrans_district,identifier2,order
0,226,3990004,399,"POLYGON ((-117.37326 33.18575, -117.37064 33.1...",0.88,0.88,0.88,1.14,1.0,68,68,0.0,0.0,11 - San Diego,1508,0
1,45,13945,3162,"POLYGON ((-118.36214 34.13853, -118.36247 34.1...",0.88,0.88,0.88,0.48,1.0,48,48,0.0,0.0,07 - Los Angeles,643,1
2,183,12825,1757,"POLYGON ((-118.28349 34.01166, -118.28350 34.0...",1.22,1.22,1.22,0.84,1.0,41,41,0.0,0.0,07 - Los Angeles,1125,2
3,183,12826,1758,"POLYGON ((-118.28343 34.01172, -118.28350 34.0...",1.22,1.22,1.22,0.88,1.0,41,41,0.0,0.0,07 - Los Angeles,43,3
4,183,8004,713,"POLYGON ((-118.26574 33.93077, -118.26564 33.9...",0.87,0.87,0.87,0.75,1.0,39,39,0.0,0.0,07 - Los Angeles,154,4
5,171,p_531943,13057,"POLYGON ((-118.15827 34.03387, -118.14975 34.0...",0.75,0.75,0.75,0.51,1.0,30,30,0.0,0.0,None,735,5
6,172,p_531943,13057,"POLYGON ((-118.15827 34.03387, -118.14975 34.0...",0.75,0.75,0.75,0.51,1.0,30,30,0.0,0.0,None,388,6
7,173,p_531943,13057,"POLYGON ((-118.15827 34.03387, -118.14975 34.0...",0.75,0.75,0.75,0.51,1.0,30,30,0.0,0.0,None,1393,7
8,174,p_531942,13057,"POLYGON ((-118.13666 34.01365, -118.13686 34.0...",0.75,0.75,0.75,0.59,1.0,30,30,0.0,0.0,None,1123,8
9,176,p_531942,13057,"POLYGON ((-118.13666 34.01365, -118.13686 34.0...",0.75,0.75,0.75,0.59,1.0,30,30,0.0,0.0,None,550,9


In [25]:
FIG_HEIGHT = 900
FIG_WIDTH = 700

'''
color_palette = (cp.CALITP_CATEGORY_BRIGHT_COLORS + cp.CALITP_CATEGORY_BOLD_COLORS + 
                 cp.CALITP_DIVERGING_COLORS +  cp.CALITP_SEQUENTIAL_COLORS
                )

random.seed(1234)
random.shuffle(color_palette)
'''
COLORSCALE = branca.colormap.step.Accent_08.scale(
    vmin = gdf.identifier2.min(),
    vmax=gdf.identifier2.max(),
)
#(colors = color_palette)

# Choose a quantitative column to plot, 
# otherwise using categorical/discrete colors will throw error
# TypeError: '<=' not supported between instances of 'str' and 'float'

PLOT_COL = "identifier2"
POPUP_DICT = {
    "calitp_itp_id": "ITP ID",
    "route_id": "Route ID",
    #"service_hours": "Bus Travel Time (hrs)",
    #"car_duration_hours": "Car Travel Time (hrs)",
    "shape_id": "shape_id",
}

# highway corridor POV
these are the top 10 corridors for oppor to do something, because they have lots of transit routes.

where are the oppor for better service on the highways
seattle has lots of hwys that are more express style, complement local transit service

what hwys have some complementary service that would benefit. 101 freeway is slow along this segment, doesn't have much. so, sort from hwy perspective, lowest first

corridor plans at caltrans

if you launched express service on hwy, where would you launch it? caltrans sponsorered express service, here is where it's beneficial.

In [26]:
m = map_utils.make_folium_choropleth_map(
    # Drop datetime, arrays cols because it's not JSON serializable
    gdf[gdf.order < 25],
    plot_col = PLOT_COL,
    popup_dict = POPUP_DICT,
    tooltip_dict = POPUP_DICT,
    colorscale = COLORSCALE,
    fig_width = FIG_WIDTH,
    fig_height = FIG_HEIGHT,
    zoom=map_utils.REGION_CENTROIDS["CA"]["zoom"]+1,
    centroid=map_utils.REGION_CENTROIDS["CA"]["centroid"],
    title="Competitive Transit Routes",
    legend_name="Legend",
)

m
#m.save(f"{utils.IMG_PATH}competitive_routes.html")

In [9]:
#m